In [123]:
from pyomo.environ import *
import pyomo.opt
import time
from time import strftime
import os

In [2]:
m = ConcreteModel()
m.bin = Var(domain=Binary)
m.b = Var(domain=PositiveReals)
m.c = Var(domain=PositiveReals)
m.d = Var(domain=Integers)
m.q = Var(domain=Reals)
m.constraint4 = Constraint(rule=lambda model: model.q - log(model.c) == 0)
m.constraint1 = Constraint(rule=lambda model: model.c + model.b <= 11)
m.constraint3 = Constraint(rule=lambda model: 3 <= model.c <= 5)
m.constrain2 = Constraint(rule=lambda model: 1 <= model.d <= 5)
m.OBJ = Objective(rule=lambda model: (model.bin * model.q) , sense=maximize)
solver = pyomo.opt.SolverFactory('bonmin', executable='/Users/ashton/school/cmsc828m/project/code/Bonmin-1.8.6.backup/bin/bonmin')
solver.solve(m, tee=True, keepfiles=True)

Solver log file: '/var/folders/bp/mm3fnf9j6fg_kwpplwd4dzyw0000gn/T/tmpbi1kn7_bonmin.log'
Solver solution file: '/var/folders/bp/mm3fnf9j6fg_kwpplwd4dzyw0000gn/T/tmpCAUoXG.pyomo.sol'
Solver problem files: ('/var/folders/bp/mm3fnf9j6fg_kwpplwd4dzyw0000gn/T/tmpCAUoXG.pyomo.nl',)
Bonmin 1.8.6 using Cbc 2.9.9 and Ipopt 3.12.8
bonmin: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT -1.6094379       10 0.004578
NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I        

{'Problem': [{'Number of objectives': 1, 'Lower bound': -inf, 'Number of variables': 5, 'Upper bound': inf, 'Sense': 'unknown', 'Number of constraints': 0}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Time': 0.15975499153137207, 'Message': 'bonmin\\x3a Optimal', 'Error rc': 0, 'Id': 3}]}

In [3]:
m.bin.value, m.c.value, m.q.value

(1.0, 5.00000004995, 1.6094379224241002)

# Real formulation

In [110]:
import pickle

In [111]:
rewards = pickle.load(open('/Users/ashton/school/cmsc828m/project/data/attacker_actions/20180429-rewards.pickle', 'rb'))
# add null option
for i in range(len(rewards)):
    for j in range(len(rewards[i])):
        rewards[i][j].append({'bs': 0, 'cve_id': 'NULL', 'es': 0, 'is': 0})
        


In [4]:
#OPTION 1
modified_rewards = []
for i in range(len(rewards)):
    for j in range(len(rewards[i])):
        for k in range(len(rewards[i][j])):
            modified_rewards.append((i,j,k))
NUM_ATTACKERS = 3
NUM_SERVICES = 4
M = 99999
LOSS_AVERSION_FACTOR = 0.1
ATTACKER_LOSS_FACTOR = 0.3
x = [0.1, 0.1, 0.1, 0.1]

In [5]:
#OPTION 2
modified_rewards = []
for i in range(2):
    for j in range(len(rewards[i])):
        for k in range(len(rewards[i][j])):
            modified_rewards.append((i,j,k))
NUM_ATTACKERS = 3
NUM_SERVICES = 2
M = 99999
LOSS_AVERSION_FACTOR = 0.1
ATTACKER_LOSS_FACTOR = 0.3
x = [0.1, 0.1, 0, 0]

In [103]:
#OPTION 3
modified_rewards = []
for i in range(len(rewards)):
    for j in range(len(rewards[i])):
        for k in range(len(rewards[i][j])):
            modified_rewards.append((i,j,k))
NUM_ATTACKERS = 3
NUM_SERVICES = len(rewards)
M = 99999
LOSS_AVERSION_FACTOR = 0.1
ATTACKER_LOSS_FACTOR = 0.3
x = [0.5 / 16] * 16

In [140]:
class Experiment:
    
    def __init__(self, attacker_prob, service_prob, rewards, pickle_path=None):
        self.attacker_prob = attacker_prob
        # changing x to self.c
        self.c = service_prob
        self.rewards = rewards
        self.pickle_path = pickle_path
        
        self.modified_rewards = []
        for i in range(len(rewards)):
            for j in range(len(rewards[i])):
                for k in range(len(rewards[i][j])):
                    self.modified_rewards.append((i,j,k))
                    
        self.m = ConcreteModel()
    
    def init_vars(self):
        self.m.attackers = Set(initialize=list(range(len(self.attacker_prob))))
        self.m.service_set = Set(initialize=list(range(len(self.c))))
        self.m.attacks = Set(initialize=self.modified_rewards, dimen=3)
        
        self.m.h = Var(self.m.service_set, domain=PositiveReals)
        self.m.n = Var(self.m.attacks, domain=Binary)
        self.m.v = Var(self.m.attackers, domain=Reals)
        self.m.q = Var(self.m.service_set, domain=PositiveReals)
    
    def set_obj(self):
        
        def obj_rule(model):
    #     return sum(attacker_prob[theta] * m.n[(s, theta, a)] * exp(m.q[s]) * (m.x_prime[s] * rewards[s][theta][a]['is']) \
    #                for s, theta, a in m.attacks)
            return sum(self.attacker_prob[theta] * model.n[(s, theta, a)] * exp(model.q[s]) * \
                   (self.c[s] * -1 * self.rewards[s][theta][a]['is'] + \
                    LOSS_AVERSION_FACTOR * model.h[s] * self.rewards[s][theta][a]['is']) \
                   for s, theta, a in model.attacks)
    
        self.m.OBJ = Objective(rule=obj_rule, sense=maximize)
    
    def set_constraints(self):
        def h_positive(model, s):
            return model.h[s] >= 0
        self.m.h_pos_constr = Constraint(self.m.service_set, rule=h_positive)

        def x_sum(model):
            return sum(model.h[s] + self.c[s] for s in model.service_set) == 1
        self.m.x_sum = Constraint(rule=x_sum)

        def best_attacker_u1(model, s, theta, a):
            #s, theta, a = attack
            return model.v[theta] - exp(model.q[s]) * (self.rewards[s][theta][a]['bs'] * self.c[s] + \
                                          -1 * ATTACKER_LOSS_FACTOR * self.rewards[s][theta][a]['es']  * model.h[s]) \
                                        >= 0.001
        #     return m.v[theta] - exp(m.q[s]) * (rewards[s][theta][a]['bs'] * x[s]) \
        #             >= -0.001
        self.m.best_attacker_constr1 = Constraint(self.m.attacks, rule=best_attacker_u1)
        
        def best_attacker_u2(model, s, theta, a):
        #s, theta, a = attack
            return model.v[theta] - exp(model.q[s]) * (self.rewards[s][theta][a]['bs'] * self.c[s] \
                                          + -1 * ATTACKER_LOSS_FACTOR * self.rewards[s][theta][a]['es']  * model.h[s]) \
                    <= (1 - model.n[(s, theta, a)]) * M + 0.001
        #     return m.v[theta] - exp(m.q[s]) * (rewards[s][theta][a]['bs'] * x[s]) \
        #             <= (1 - m.n[(s, theta, a)]) * M + .001
        self.m.best_attacker_constr2 = Constraint(self.m.attacks, rule=best_attacker_u2)           

        def only_one_action(model, attacker):
            return sum(model.n[(s, theta, a)] for s, theta, a in model.attacks if theta == attacker) == 1

        self.m.only_one_action_constr = Constraint(self.m.attackers, rule=only_one_action)

        def q_rule(model, s):
            return model.q[s] == -1 * log(self.c[s] + model.h[s])
        self.m.q_constr = Constraint(self.m.service_set, rule=q_rule)
        
    # used to simulate difference between believed attacker probabilities and "real" attacker probabilities
    def real_attacker_obj(self, model, real_attacker_probs):
        return sum(real_attacker_probs[theta] * model.n[(s, theta, a)].value * exp(model.q[s].value) * \
               (self.c[s] * -1 * self.rewards[s][theta][a]['is'] + \
                LOSS_AVERSION_FACTOR * model.h[s].value * self.rewards[s][theta][a]['is']) \
               for s, theta, a in model.attacks)
    
    
    
    def solve(self, tee_p=False):
        self.solver = pyomo.opt.SolverFactory('bonmin', executable='/Users/ashton/school/cmsc828m/project/code/Bonmin-1.8.6.backup/bin/bonmin')
        start_time = time.time()
        self.solver.solve(self.m, tee=tee_p)
        end_time = time.time()
        self.duration = end_time - start_time
       

    
    # gets the csv log itself
    def get_log(self):
        return ",".join([
            str(len(self.c)),
            str(self.c),
            str(self.m.h),
            str(self.duration),
            self.pickle_path
        ])
    
    def print_results(self):
        print("Attacks")
        print("(service, attacker, attack), was_selected, CVE Info")
        for n_i in self.m.n:
            if self.m.n[n_i].value > 0:
                s, theta, a = n_i
                print (n_i, self.m.n[n_i].value, self.rewards[s][theta][a])
                
        print()
        print("Obj: ", self.m.OBJ())
        
        print()
        print("Service Honeypot Distribution:")
        for s in self.m.service_set:
            print((s, self.m.h[s].value))
        
# gets the header for the csv log
def get_log_header():
    return ",".join([
        "num_services",
        "c",
        "x",
        "duration",
        "experiment_pickle_path"
    ]) + "\n"

In [99]:
# removing attacks which are strictly dominated 
new_rewards = []
for s in range(len(rewards)):
    new_rewards.append([])
    for theta in range(len(rewards[s])):
        new_rewards[s].append([])
        already_added = []
        for a_1 in rewards[s][theta]:
            if not any(a_2['bs'] >= a_1['bs'] and a_2['es'] < a_1['es'] for a_2 in rewards[s][theta]) and \
                    not (a_1['bs'], a_1['es']) in already_added:
                new_rewards[s][theta].append(a_1)
                already_added.append((a_1['bs'], a_1['es']))

In [107]:
e = Experiment(attacker_prob=[0.5, 0.4, 0.1], service_prob=x, rewards=rewards)
e.init_vars()
e.set_obj()
e.set_constraints()
e.solve()


In [48]:
rewards = rewards[:2]
e = Experiment(attacker_prob=[0.5, 0.4, 0.1], service_prob=[0.1, 0.2], rewards=rewards)
e.init_vars()
e.set_obj()
e.set_constraints()
e.solve()

In [108]:
e.m.OBJ()

-2.6948705633642187

In [50]:
e.real_attacker_obj(e.m, [0.99, 0, .01])

-0.3422945896273375

In [109]:
e.print_results()

Attacks
(service, attacker, attack), was_selected, CVE Info
((6, 0, 0), 1.0, {'is': 2.9, 'es': 4.9, 'cve_id': u'CVE-2015-3455', 'bs': 2.6})
((7, 1, 1), 1.0, {'is': 2.9, 'es': 8.0, 'cve_id': u'CVE-2014-0229', 'bs': 4.0})
((6, 2, 3), 1.0, {'is': 6.9, 'es': 10.0, 'cve_id': u'CVE-2013-1839', 'bs': 7.8})
()
('Obj: ', -2.6948705633642187)
()
Service Honeypot Distribution:
(0, 0.04379732253746351)
(1, 0.04813397296842657)
(2, 0.04382969909476854)
(3, 0.027339576367764788)
(4, 0.04382969909476854)
(5, 0.027339576371402392)
(6, 0.004139796508672447)
(7, 0.010890707519950228)
(8, 0.02931450590812864)
(9, 0.026021421321729864)
(10, 0.027339576367764806)
(11, 0.047448767741180925)
(12, 0.04379732253746347)
(13, 0.0475942666603483)
(14, 0.01134436275406703)
(15, 0.01783942624610001)


In [142]:
# experiment running
def run_experiments(experiments, experiments_dir):
    # create dir
    new_dir_name = time.strftime("%Y-%m-%d_%H-%M-%S", time.gmtime(time.time()))
    os.mkdir(experiments_dir + new_dir_name)
    new_dir_path = os.path.join(experiments_dir, new_dir_name)
    
    # create file
    with open(os.path.join(new_dir_path, "results.csv"), 'w') as f:
        # write header
        f.write(get_log_header())
        for i, e in enumerate(experiments):
            # run experiment
            e.init_vars()
            e.set_obj()
            e.set_constraints()
            e.solve()
            # write pickle
            e.pickle_path = os.path.join(new_dir_path, str(i) + ".pickle")
            pickle.dump(e, open(e.pickle_path, 'w'))
            # write line to file
            f.write(e.get_log() + '\n')

experiments = [Experiment(attacker_prob=[0.5, 0.4, 0.1], service_prob=[0.1], rewards=[rewards[0]]),
              Experiment(attacker_prob=[0.5, 0.4, 0.1], service_prob=[0.3], rewards=[rewards[0]])]
run_experiments(experiments, "/tmp/")

In [118]:
time.strftime("%Y-%m-%d_%H-%M-%S", time.gmtime(time.time()))

'2018-04-30_01-58-19'